# Data analysis
## September 2024

* Step 1: import the data set
* Step 2: find the number of segmentations, and their size (number of pixels per slice, so we can get 3D volume)
* Step 3: Overlay the segmentations on the other channels
* Step 4: Get the intensity in each channel for the segmentations
* Step 5: Save it out



In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams["image.interpolation"] = 200
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize
from csbdeep.io import save_tiff_imagej_compatible

from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist import Rays_GoldenSpiral
from stardist.matching import matching, matching_dataset


from skimage.segmentation import clear_border

import seaborn as sns
import matplotlib.pyplot as plt


import os

from skimage.measure import label, regionprops, regionprops_table


import numpy as np
import pandas as pd


C:\Users\fallest\AppData\Local\Temp\ipykernel_45572\1024161798.py:6: MatplotlibDeprecationWarning: Support for setting an rcParam that expects a str value to a non-str value is deprecated since 3.5 and support will be removed two minor releases later.
  matplotlib.rcParams["image.interpolation"] = 200


### Step 1: Import image

In [2]:
Image_path = "C:\\3D_Segmentation\\3d_monolayer_xy1_ch2.tif"
mask_path = "C:\\3D_Segmentation\\output\\Cellpose\\3d_monolayer_xy1_ch2_Seg_Type_Stitching.tif"
save_directory = "C:\\3D_Segmentation\\output\\"

In [3]:
segmentations = imread(mask_path)
intensity_image = imread(Image_path)

In [4]:
#If the voxels are anisotropic, change this here from 1's, to represent what they are
X_ani = 1
Y_ani = 1
Z_ani = 1

## Make some functions useful for our calculations

In [5]:

def count_pixels_for_label(segmentation_array, target_label):
    
    """
    Count the number of pixels for a specific label in each slice of a 3D segmentation array.
    Only include slices where the count of pixels for the target label is greater than zero.
    
    Parameters:
    - segmentation_array (np.ndarray): A 3D numpy array where each slice represents a segmentation map.
    - target_label (int): The label for which pixel counts are to be computed.
    
    Returns:
    - List[Tuple[int, int]]: A list of tuples where each tuple contains the slice number and the count of pixels for the target label.
    """
    
    # Initialize a list to store counts of the target label for each slice along with slice index
    pixel_counts = []

    # Iterate through each slice
    for slice_index in range(segmentation_array.shape[0]):
        slice_data = segmentation_array[slice_index]
        
        # Count the number of pixels for the target label in this slice
        count = np.sum(slice_data == target_label)
        
        # Only append to the list if count is greater than zero
        if count > 0:
            pixel_counts.append((slice_index, count))
    

    return pixel_counts

def volume_of_labels(segmentation_array, x_size=1, y_size=1, z_size=1):

    volume_dict={}
    total_volume_dict = {}
    for label in np.unique(segmentation_array):
        if label>0:
            total_volume=0
            volume_dict[label]=count_pixels_for_label(segmentation_array, label)
            for slice in volume_dict[label]:
                slice_volume = slice[1]*x_size*y_size*z_size
                total_volume = total_volume+slice_volume
            total_volume_dict[label]=total_volume
    

    return volume_dict, total_volume_dict

def get_z_slices(df, segmentation_array):
    total_slices = {}
    for target_label in df['label']:
        slice_count = np.sum(np.any(segmentation_array == target_label, axis=(1, 2)))
        total_slices[target_label]=slice_count

    return total_slices



In [6]:
#Use region props to get the properties of the intensity image wherever there are labels in the label image
#The intensity image can be any of the channels from the image
data_dict =regionprops_table(segmentations, intensity_image = intensity_image, properties=('label', 'area', 'slice', 'image_intensity', 'intensity_max', 'intensity_mean', 'intensity_min'))

In [7]:
data_dict['total_intensity'] = [sum(x[~np.isnan(x)]) if isinstance(x, np.ndarray) else 0 for x in data_dict['image_intensity']]   #sum the total intensity of the object across all slices 
total_slices = get_z_slices(data_dict, segmentations) #get the number of slices for doing volume
total_slices = dict(sorted(total_slices.items()))
total_slices = [total_slices[key] for key in sorted(total_slices)]
data_dict.pop('image_intensity') #get rid of image intensity, as that's per slice, and we want the whole total image
data_dict.pop('slice') #get rid of slice number

volume_dict, total_volumes = volume_of_labels(segmentations, X_ani, Y_ani, Z_ani) #calculate the Volume

data_frame = pd.DataFrame(data_dict) #make a dataframe
data_frame['total_volume'] = data_frame['label'].map(total_volumes)


In [8]:
data_frame

,label,area,intensity_max,intensity_mean,intensity_min,total_intensity,total_volume
0,1,75188,170.0,112.611520,99.0,8467035,75188
1,2,1245,112.0,105.240161,101.0,131024,1245
2,3,913,111.0,105.136911,101.0,95990,913
3,4,2415,111.0,105.866667,102.0,255668,2415
4,5,79641,132.0,108.285130,99.0,8623936,79641
5,6,1684,111.0,105.258907,101.0,177256,1684
6,7,11574,130.0,106.545274,100.0,1233155,11574
7,8,76375,140.0,111.390756,100.0,8507469,76375
8,9,52091,159.0,115.511086,100.0,6017088,52091
9,10,76826,155.0,112.692877,100.0,8657743,76826


In [10]:
save_path = save_directory+os.path.basename(Image_path)[:-4]+"_data.csv"
os.makedirs(save_path, exist_ok=True) if not os.path.exists(save_path) else None
print(save_path)
data_frame.to_csv(save_path, index=False)

C:\3D_Segmentation\output\3d_monolayer_xy1_ch2_data.csv
